In [ ]:
%pip install transformers bitsandbytes accelerate datasets

# Generative LLMs for Text Classification
Here, we implement a simple pipeline to use Generative LLMs for generic text classification tasks. We will focus on explainability, directly asking the model to explain its answers.

In [11]:
import torch
from transformers import pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"

## Model
Let's load the LLM from Hugging Face. We will use Google's Gemma 3 (1B) instruction tuned model. We load the Unsloth version since it doesn't require user authentication, but it's the same model.

In [ ]:
llm = pipeline("text-generation", model="unsloth/gemma-3-1b-it", device=device)

## Prompts
Let's define different prompts to see how they affect the model's behaviour. We will start with a **zero-shot** prompt, i.e., asking the instruction tuned model to perform the task without any examples or fine-tuning.

In [21]:
zero_shot_prompt = """Classify the emotion of this Text:

"{text}"

Only use one of the following classes: ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"].
Classify the emotion using the "EMOTION" tag and provide a concise explanation using the "EXPLANATION" tag. Pay particular attention to sarcasm and irony when classifying."""

Now let's write a **few-shot** prompt, i.e., in addition to asking the model to perform the task, we include a few examples of the task being performed.

In [22]:
few_shot_prompt = """Classify the emotion of this Text:

"{text}"

Only use one of the following classes: ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"].
Classify the emotion using the "EMOTION" tag and provide a concise explanation using the "EXPLANATION" tag. Pay particular attention to sarcasm and irony when classifying.

Here are some Examples:

Text: "I can't believe they lied to me again. I'm done with them."
EMOTION: anger
EXPLANATION: The speaker expresses frustration and a sense of betrayal, which are indicators of anger.

Text: "I just got promoted at work today! I'm so happy!"
EMOTION: joy
EXPLANATION: The speaker is celebrating a positive personal achievement, showing signs of joy.

Text: "She didn’t show up again... I guess I should stop expecting anything."
EMOTION: sadness
EXPLANATION: The speaker feels let down and emotionally hurt, suggesting a mood of sadness.

Text: "Oh great, another meeting that could’ve been an email. Just what I needed today."
EMOTION: disgust
EXPLANATION: The sarcastic tone conveys annoyance and disdain, which are characteristic of disgust."""

## Test
Now we can test the model and how it performs with the different prompting strategies.

In [34]:
text = "Seriously? Another delay? This is so frustrating."

zero_shot_messages = [{"role": "user", "content": [{"type": "text", "text": zero_shot_prompt.format(text=text)}]}]
few_shot_messages = [{"role": "user", "content": [{"type": "text", "text": few_shot_prompt.format(text=text)}]}]

zero_shot_out = llm(zero_shot_messages, max_new_tokens=50)
few_shot_out = llm(few_shot_messages, max_new_tokens=50)

Let's inspect the results.

In [35]:
print(f"Text: {text}\n")
print('# ZERO-SHOT')
print(zero_shot_out[0]['generated_text'][1]['content'])
print('\n# FEW-SHOT')
print(few_shot_out[0]['generated_text'][1]['content'])

Text: Seriously? Another delay? This is so frustrating.

# ZERO-SHOT
Classify the emotion: **disgust**

EXPLANATION: The phrase “Seriously? Another delay?” combined with the exclamation "This is so frustrating" creates a tone of deep disapproval and annoyance. The use of “Seriously?” subtly implies a

# FEW-SHOT
Text: "It’s just… a really, really long wait. I'm starting to think I'll be sleeping on the job."

EMOTION: frustration
EXPLANATION: The speaker expresses a feeling of annoyance and helplessness


Let's try to add a **system prompt** to make it easier for the model to follow the instruction.

In [37]:
categories = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]

system_prompt = f"""You calssify texts into one of the following pre-defined categories: {categories}.
You output the classification with the following format: CLASSIFICATION: <category>
and provide a brief, concise explanation with the following format: EXPLANATION: <explanation>.
The list of category is **exhaustive**."""

zero_shot_prompt = "Text: \"{text}\""

system_messages = [
    {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
    {"role": "user", "content": [{"type": "text", "text": zero_shot_prompt.format(text=text)}]},
]

system_out = llm(system_messages, max_new_tokens=50)

Let's inspect the result.

In [40]:
print(system_out[0]['generated_text'][2]['content'])

CLASSIFICATION: anger
EXPLANATION: The phrase expresses frustration and annoyance with a significant delay.
